In [2]:
#pip install vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import altair as alt
import os 
from vega_datasets import data

In [4]:
cuisine_mapping = pd.read_csv('data/Cuisinne Mapping.csv')

In [5]:
# List files in data/Taste/
files = os.listdir('data/Taste/')
# Read in all files with pandas
# Filename as a column
dfs = []
for file in files:
    dfs.append(pd.read_csv('data/Taste/' + file))
    
# Concatenate all files
taste = pd.concat(dfs)

In [6]:
# Merge with cuisine mapping
mapping = taste.merge(cuisine_mapping, on='cuisine')
# Filter out those with blank "ISO 31661 Numeric"
mapping_filtered = mapping[mapping['ISO 31661 Numeric'].notnull()]
mapping_filtered.loc[:,'id'] = mapping_filtered['ISO 31661 Numeric'].astype(int).astype(str)

C:\Users\msuryana\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [7]:
mapping_filtered.head()

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id,title,cuisine,Country,ISO 31661 Numeric,Notes
170,100.00,44.25,97.58,38.63,22.14,44.58,8200.0,840,Jack Burgers,American,United States,840.0,NaN
171,100.00,48.26,10.31,2.98,29.82,53.99,0.0,840,Sweet Potato Lentil Chili,American,United States,840.0,NaN
172,8.56,3.64,0.00,0.00,0.00,100.00,160000.0,840,Black Bean-Quinoa Burgers,American,United States,840.0,NaN
173,98.80,60.79,10.53,27.40,33.21,100.00,0.0,840,Healthy Low Fat Burgers,American,United States,840.0,NaN
174,100.00,3.65,0.00,0.00,0.00,43.50,0.0,840,Dill Salmon Burgers,American,United States,840.0,NaN


In [122]:
source = alt.topo_feature(data.world_110m.url, 'countries')
sphere = alt.sphere()
graticule = alt.graticule()

# flavors = mapping_filtered.columns[0:7]
# input_dropdown = alt.binding_select(options=flavors, name='Flavor')
# selection = alt.selection_single(fields=['Origin'], bind=input_dropdown)

flavor = 'bitterness'

# Layering and configuring the components
alt.Chart(source).mark_geoshape(
    stroke='black'
    ).encode(
        color= flavor + ':Q'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(mapping_filtered, 'id', [flavor])
).project(
    'naturalEarth1'
).properties(width=600, height=400).configure_view(stroke=None)

alt.Chart(...)

In [12]:
# Same as above, but with a filter for the flavor

# Keep first 8 columns of mapping_filtered
mapping_filtered_flavor = mapping_filtered[mapping_filtered.columns[0:8]]




In [14]:
mapping_filtered_flavor

,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness,id
170,100.00,44.25,97.58,38.63,22.14,44.58,8200.0,840
171,100.00,48.26,10.31,2.98,29.82,53.99,0.0,840
172,8.56,3.64,0.00,0.00,0.00,100.00,160000.0,840
173,98.80,60.79,10.53,27.40,33.21,100.00,0.0,840
174,100.00,3.65,0.00,0.00,0.00,43.50,0.0,840
...,...,...,...,...,...,...,...,...
4165,60.83,100.00,30.19,48.82,57.92,72.48,5950.0,250
4166,27.99,63.69,96.70,84.22,31.66,100.00,20000.0,250
4167,67.87,76.07,88.61,45.56,42.43,100.00,600000.0,250
4168,37.53,100.00,44.21,32.13,34.76,92.52,300000.0,250


In [17]:
mapping_filtered_flavor_pivoted = pd.melt(mapping_filtered_flavor, id_vars=['id'], value_vars=['sweetness', 'saltiness','sourness','bitterness','savoriness','fattiness', 'spiciness'])
# Melt with 'id' as key

In [18]:
# Take average of value by id, variabl
averaged = mapping_filtered_flavor_pivoted.groupby(['id','variable']).mean()
# Reset hierarchical index
averaged = averaged.reset_index()

In [58]:
averaged

,id,variable,value
0,156,bitterness,3.345268e+01
1,156,fattiness,6.659982e+01
2,156,saltiness,8.688942e+01
3,156,savoriness,5.350402e+01
4,156,sourness,2.619758e+01
5,156,spiciness,8.045560e+05
6,156,sweetness,4.682408e+01
7,250,bitterness,3.116740e+01
8,250,fattiness,7.055514e+01
9,250,saltiness,8.807460e+01


In [136]:
## NOT WORKING - needs debugged (only showing spiciness for some reason)
input_dropdown = alt.binding_select(options=list(mapping_filtered_flavor.columns[0:7]), name='Flavor')
selection = alt.selection_single(fields=['variable'], bind=input_dropdown)

# Create same map as above but with a filter for the flavor
# Layering and configuring the components
chart = alt.Chart(source).mark_geoshape(
    ).encode(
        color= 'value:Q',
        # opacity = 'variable:N'
    ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(averaged, 'id', ['value','variable'])
).project(
    'equirectangular'
).properties(width=600, height=400
).configure_view(stroke=None
).add_selection(
    selection
).transform_filter(selection)
chart

alt.Chart(...)

In [158]:
#flavor_pivoted_byCuisine = pd.melt(taste, id_vars=['cuisine'], value_vars=['sweetness', 'saltiness','sourness','bitterness','savoriness','fattiness', 'spiciness'])
# Melt with 'cuisine' as key
import numpy as np

flavor_grouped_byCuisine = taste.groupby("cuisine", as_index=False).agg(
    sweetness = ("sweetness", np.mean),
    saltiness = ("saltiness", np.mean),
    sourness = ("sourness", np.mean),
    bitterness = ("bitterness", np.mean),
    savoriness = ("savoriness", np.mean),
    fattiness = ("fattiness", np.mean),
    spiciness = ("spiciness", np.mean)
)

In [159]:
flavor_grouped_byCuisine

,cuisine,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness
0,African,62.137824,67.217353,34.865824,28.495176,38.197941,66.10300,5.527374e+05
1,American,59.044920,72.916520,41.882140,30.361040,43.979340,67.00252,1.446679e+06
2,British,66.256100,44.988180,23.402340,17.131300,23.229080,74.42786,6.065368e+04
3,Cajun,46.537140,77.776140,34.270740,26.874960,46.474800,68.57418,2.792083e+05
4,Caribbean,62.250940,60.223000,25.631920,22.126660,39.433700,60.23418,6.899148e+05
5,Chinese,46.824080,86.889420,26.197580,33.452680,53.504020,66.59982,8.045560e+05
6,Eastern European,39.229620,95.212700,32.917500,29.876300,58.345860,63.77872,1.999277e+05
7,European,54.305180,78.691680,46.642340,36.208560,45.267140,68.12250,1.050325e+05
8,French,44.684660,88.074600,34.860460,31.167400,44.656160,70.55514,1.559065e+05


In [161]:
## Create bar chart with flavor profiling
## NEED TO UPDATE FOR MULTIPLE CUISINE SELECTION
## NEED TO UPDATE TO DYNAMICALLY SPIT OUT MULTIPLE CHARTS
input_dropdown_cuisine = alt.binding_select(options=taste.cuisine.unique(), name='Cuisines')
selection_cuisine = alt.selection_single(fields=['cuisine'], bind=input_dropdown_cuisine)
alt.data_transformers.enable('default', max_rows=None)
# Create same map as above but with a filter for the flavor
# Layering and configuring the components
chart_sweet = alt.Chart(flavor_grouped_byCuisine).mark_bar(
    ).encode(
        x= 'sweetness:Q',
    y = 'cuisine'
    ).properties(width=600, height=400
).add_selection(
    selection_cuisine
).transform_filter(selection_cuisine)

chart_sweet

alt.Chart(...)